### Purpose & Data Sources  

**Project goal**  
This notebook performs the **integration phase** of our TFT analytics pipeline.  
It combines in-game match data (`tft_matches_flat.csv`) with static metadata from **Riot Games Data Dragon** to create a unit-level table ready for downstream cleaning and analysis.

| Aspect | Details |
|--------|---------|
| **Raw match data** | `curated/tft_matches_flat.csv` (✧ 359 K rows, exported from Riot TFT API). |
| **Metadata source** | Data Dragon v **15.7.1** (champions / items JSON only). |
| **Integration logic** | 1⃣ explode the `units` string → one row per unit<br>2⃣ normalize TFT-specific unit names (regex)<br>3⃣ left-join on champion key (`id`) to attach `championId`, `name`, `title`, etc. |
| **Output artifacts** | `tft_units_long.parquet` or `*.csv.gz` (auto-fallback) <br><sup>• 3 591 rows × 11 columns</sup> |
| **Unmatched rate** | 791 units (~22 %) could not be mapped to Data Dragon (see “Unmatched champions” section below). |

> Subsequent cleaning steps — missing-value imputation, outlier checks, duplicate handling, and rule-based repairs — are implemented in **`02_cleaning.ipynb`**.


In [2]:
# Imports and project paths
from pathlib import Path
import tarfile, json, requests, pandas as pd

# Define project paths
PROJ_ROOT = Path.cwd().parent
RAW_DIR = PROJ_ROOT / "raw"
CURATED_DIR = PROJ_ROOT / "curated"
RAW_DIR.mkdir(exist_ok=True)

# Version setting
VERSION = "15.7.1"
TGZ_URL = f"https://ddragon.leagueoflegends.com/cdn/dragontail-{VERSION}.tgz"
TGZ_PATH = RAW_DIR / f"ddragon_{VERSION}.tgz"
DD_JSON_DIR = RAW_DIR / f"dd_{VERSION}"


In [3]:
# Download Data Dragon tarball if not already downloaded
if not TGZ_PATH.exists():
    print("Downloading Data Dragon...")
    r = requests.get(TGZ_URL, stream=True)
    r.raise_for_status()
    TGZ_PATH.write_bytes(r.content)
else:
    print("TGZ file already exists.")

TGZ file already exists.


In [4]:
# Extract JSON files only from en_US directory
if not DD_JSON_DIR.exists():
    DD_JSON_DIR.mkdir()
    with tarfile.open(TGZ_PATH, "r:gz") as tar:
        members = [
            m for m in tar.getmembers()
            if "data/en_US/" in m.name and m.name.endswith(".json")
        ]
        tar.extractall(path=DD_JSON_DIR, members=members)
else:
    print("JSON files already extracted.")


JSON files already extracted.


In [5]:
# Locate champion.json and item.json even if nested under an extra version folder
champ_file = next(DD_JSON_DIR.rglob("data/en_US/champion.json"))
item_file  = next(DD_JSON_DIR.rglob("data/en_US/item.json"))

# === Champion reference table =================================================
with open(champ_file) as f:
    champ_raw = json.load(f)["data"]          # dict keyed by champName, e.g. {"Aatrox": {...}}
    
champ_df = (
    pd.json_normalize(champ_raw.values())     # flatten each champ dict
      .assign(championId=lambda d: d["key"].astype(int))  # "key" is numeric string
      .loc[:, ["championId", "name", "id", "title"]]      # keep only needed cols
)

# === Item reference table =====================================================
with open(item_file) as f:
    item_raw = json.load(f)["data"]           # dict keyed by itemId, e.g. {"1001": {...}}

# build DataFrame: bring the dict key (itemId) back as a column
item_df = (
    pd.DataFrame.from_dict(item_raw, orient="index")  # index == itemId str
      .rename_axis("itemId")            # move index to a real column
      .reset_index()
      .assign(itemId=lambda d: d["itemId"].astype(int))  # convert to int
)

# optional: flatten nested 'gold' dict so you get 'gold.total' etc.
gold_cols = pd.json_normalize(item_df["gold"]).add_prefix("gold.")
item_df = pd.concat([item_df.drop(columns=["gold"]), gold_cols], axis=1)

# keep only useful columns
item_df = item_df.loc[:, ["itemId", "name", "plaintext", "gold.total", "stats"]]

In [6]:
item_df

,itemId,name,plaintext,gold.total,stats
0,1001,Boots,Slightly increases Move Speed,300,{'FlatMovementSpeedMod': 25}
1,1004,Faerie Charm,Slightly increases Mana Regen,200,{}
2,1006,Rejuvenation Bead,Slightly increases Health Regen,300,{}
3,1011,Giant's Belt,Greatly increases Health,900,{'FlatHPPoolMod': 350}
4,1018,Cloak of Agility,Increases critical strike chance,600,{'FlatCritChanceMod': 0.15}
...,...,...,...,...,...
610,9404,Wandering Storms,,0,{}
611,9405,Grizzly Smash,,0,{}
612,9406,Lover's Ricochet,,0,{}
613,9407,Hopped-Up Hex,,0,{}


In [ ]:
# Read flat match data from CSV
match_path = CURATED_DIR / "tft_matches_flat.csv"
df = pd.read_csv(match_path)

print("Loaded data shape:", df.shape)
df.head()


Loaded data shape: (400, 11)


,match_id,game_time,patch,queue_id,puuid,placement,level,gold_left,status,traits,units
0,NA1_5260355615,2025-04-05 16:02:41.389000+00:00,15.7,1100,-E45uMGggx3vOTDyHOknqk5WpCIL9M5_8fgi2E3ETtzBGV...,8,8,1,ok,"TFT14_Armorclad-1,TFT14_BallisTek-0,TFT14_Brui...","Jax-2,Poppy-2,Jhin-1,Naafiri-1,Varus-1,Mordeka..."
1,NA1_5260355615,2025-04-05 16:02:41.389000+00:00,15.7,1100,03TxmgwS2Z7v6szM5XPTsSemWtTByCU1t04yeq9fmYy0N3...,7,8,1,ok,"TFT14_Armorclad-2,TFT14_Bruiser-0,TFT14_Cutter...","Kindred-1,Jax-1,Shyvana-1,Varus-2,Mordekaiser-..."
2,NA1_5260355615,2025-04-05 16:02:41.389000+00:00,15.7,1100,haxrw5LntpypScQN1t_8FmGAfB6-M0CtkfQqkXtE3fVVjQ...,6,8,4,ok,"TFT14_AnimaSquad-3,TFT14_Armorclad-1,TFT14_Con...","Sylas-2,Vayne-1,Illaoi-1,Yuumi-2,Xayah-2,Leona..."
3,NA1_5260355615,2025-04-05 16:02:41.389000+00:00,15.7,1100,rC9fkNsc87GwPDWnjQugEwcbmONnmQM7nKHt9O1thwtLKe...,5,8,47,ok,"TFT14_AnimaSquad-1,TFT14_Divinicorp-1,TFT14_Ed...","Jhin-2,Rhaast-2,Braum-2,Jarvan-2,Xayah-2,Leona..."
4,NA1_5260355615,2025-04-05 16:02:41.389000+00:00,15.7,1100,zLXLHSmuSapUIaPhYvGf61Il_wXMm6aVKM4BnmFYRBKLdE...,4,8,45,ok,"TFT14_Armorclad-1,TFT14_Bruiser-0,TFT14_EdgeRu...","NidaleeCougar-3,Kindred-3,Shyvana-3,Jhin-3,Naa..."


Helper functions

In [11]:
import re
from pathlib import Path

def normalize_tft_champion(raw: str) -> str:
    """
    Convert TFT-specific unit strings to base champion names.
    Examples:
        'TFT14_Zeri'    -> 'Zeri'
        'NidaleeCougar' -> 'Nidalee'
        'Rhaast'        -> 'Kayn' (keep Kayn, drop form)
    """
    raw = re.sub(r"^TFT\d+_", "", raw)              # strip TFT patch prefix
    raw = re.sub(r"(Cougar|Rhaast|Dragon|Mega)$", "", raw)  # strip forms
    return raw

def safe_save(df, out_base: Path, index: bool = False):
    """
    Save DataFrame to <name>.parquet if a parquet engine is available;
    otherwise save to <name>.csv.gz. Return the actual path.
    """
    parquet_path = out_base.with_suffix(".parquet")
    csv_gz_path  = out_base.with_suffix(".csv.gz")
    try:
        df.to_parquet(parquet_path, index=index)
        print(f"Saved as Parquet: {parquet_path}")
        return parquet_path
    except ImportError:
        print("PyArrow/Fastparquet not found – falling back to gzip-CSV.")
        df.to_csv(csv_gz_path, index=index, compression="gzip")
        print(f"Saved as gzip-CSV: {csv_gz_path}")
        return csv_gz_path

Explode `units` and merge with champion metadata

In [12]:
units_long = (
    df[["match_id", "puuid", "units"]]
      .dropna(subset=["units"])
      .assign(unit_list=lambda d: d["units"].str.split(","))
      .explode("unit_list")
      .assign(unit_list=lambda s: s["unit_list"].str.strip())
)

# Split "<Champ>-<Star>"
units_long[["champRaw", "star_level"]] = (
    units_long["unit_list"].str.rsplit("-", n=1, expand=True)
)
units_long["star_level"] = units_long["star_level"].astype(int)

# Normalize champion name and merge
units_long["championKey"] = units_long["champRaw"].apply(normalize_tft_champion)
units_merged = units_long.merge(
    champ_df,
    how="left",
    left_on="championKey",
    right_on="id",
    suffixes=("", "_champ")
)

print("Units long table shape:", units_merged.shape)
print("Unmatched champions:", units_merged['name'].isna().sum())

Units long table shape: (3591, 11)
Unmatched champions: 791


In [15]:
units_merged

,match_id,puuid,units,unit_list,champRaw,star_level,championKey,championId,name,id,title
0,NA1_5260355615,-E45uMGggx3vOTDyHOknqk5WpCIL9M5_8fgi2E3ETtzBGV...,"Jax-2,Poppy-2,Jhin-1,Naafiri-1,Varus-1,Mordeka...",Jax-2,Jax,2,Jax,24.0,Jax,Jax,Grandmaster at Arms
1,NA1_5260355615,-E45uMGggx3vOTDyHOknqk5WpCIL9M5_8fgi2E3ETtzBGV...,"Jax-2,Poppy-2,Jhin-1,Naafiri-1,Varus-1,Mordeka...",Poppy-2,Poppy,2,Poppy,78.0,Poppy,Poppy,Keeper of the Hammer
2,NA1_5260355615,-E45uMGggx3vOTDyHOknqk5WpCIL9M5_8fgi2E3ETtzBGV...,"Jax-2,Poppy-2,Jhin-1,Naafiri-1,Varus-1,Mordeka...",Jhin-1,Jhin,1,Jhin,202.0,Jhin,Jhin,the Virtuoso
3,NA1_5260355615,-E45uMGggx3vOTDyHOknqk5WpCIL9M5_8fgi2E3ETtzBGV...,"Jax-2,Poppy-2,Jhin-1,Naafiri-1,Varus-1,Mordeka...",Naafiri-1,Naafiri,1,Naafiri,950.0,Naafiri,Naafiri,the Hound of a Hundred Bites
4,NA1_5260355615,-E45uMGggx3vOTDyHOknqk5WpCIL9M5_8fgi2E3ETtzBGV...,"Jax-2,Poppy-2,Jhin-1,Naafiri-1,Varus-1,Mordeka...",Varus-1,Varus,1,Varus,110.0,Varus,Varus,the Arrow of Retribution
...,...,...,...,...,...,...,...,...,...,...,...
3586,NA1_5191565231,lA9GyGpJeiE61z2lTsAsGI-d7oT3ppesOTpkAHx-OJ7rZ_...,"Lux-2,Vladimir-2,swain-2,Nami-2,elise-2,DrMund...",DrMundo-2,DrMundo,2,DrMundo,36.0,Dr. Mundo,DrMundo,the Madman of Zaun
3587,NA1_5191565231,lA9GyGpJeiE61z2lTsAsGI-d7oT3ppesOTpkAHx-OJ7rZ_...,"Lux-2,Vladimir-2,swain-2,Nami-2,elise-2,DrMund...",Zoe-2,Zoe,2,Zoe,142.0,Zoe,Zoe,the Aspect of Twilight
3588,NA1_5191565231,lA9GyGpJeiE61z2lTsAsGI-d7oT3ppesOTpkAHx-OJ7rZ_...,"Lux-2,Vladimir-2,swain-2,Nami-2,elise-2,DrMund...",LeBlanc-1,LeBlanc,1,LeBlanc,NaN,NaN,NaN,NaN
3589,NA1_5191565231,lA9GyGpJeiE61z2lTsAsGI-d7oT3ppesOTpkAHx-OJ7rZ_...,"Lux-2,Vladimir-2,swain-2,Nami-2,elise-2,DrMund...",Mordekaiser-1,Mordekaiser,1,Mordekaiser,82.0,Mordekaiser,Mordekaiser,the Iron Revenant


Champion Metadata Integration Summary
Units long table shape:
(3,591 rows, 11 columns)
Each row represents a single unit placed by a player in a match. A total of 3,591 unit-level records were generated after exploding the units column.

Unmatched champions:
791 rows (~22% of total) could not be matched to official champion metadata from Data Dragon. These rows have missing values in fields such as championId, name, and title.

❓ Why are some champions unmatched?
Special unit variations

Examples: LeBlanc2, SwainDragon, EliseSpider, KaynRhaast, etc. These are alternate forms or transformed units used in TFT that don't exactly match base champion names in Data Dragon.

Outdated Data Dragon version

The current version (e.g., 15.7.1) may not include newly released champions or updated names used in recent TFT patches.

Non-standard units

Some units may represent temporary summoned entities, PvE mobs, or test units not included in the base champion list.

Save long-format unit table


In [13]:
units_out_base = CURATED_DIR / "tft_units_long"
units_out_path = safe_save(units_merged, units_out_base, index=False)

PyArrow/Fastparquet not found – falling back to gzip-CSV.
Saved as gzip-CSV: /home/ericlin/tft/curated/tft_units_long.csv.gz


In [14]:
wide_units = (
    units_merged
      .groupby(["match_id", "puuid"])
      .agg({"championId": list, "star_level": list})
      .reset_index()
)

wide_out_base = CURATED_DIR / "tft_units_wide"
wide_out_path = safe_save(wide_units, wide_out_base, index=False)

PyArrow/Fastparquet not found – falling back to gzip-CSV.
Saved as gzip-CSV: /home/ericlin/tft/curated/tft_units_wide.csv.gz
